In [ ]:
# A/B TESTING LAB 

import numpy as np
import pandas as pd
from pathlib import Path

from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

# 1) Load CSV or Generate Data
csv_path = Path("ab_test_data.csv")

if csv_path.exists():
    df = pd.read_csv(csv_path)
    print("✅ Loaded:", csv_path)
else:
    print("⚠️ CSV not found. Generating dataset...")

    np.random.seed(42)
    n = 10000

    # Random assignment to groups (50/50)
    group = np.random.choice(['A', 'B'], size=n, p=[0.5, 0.5])

    # Base conversion rates
    conv_rate_A = 0.05
    conv_rate_B = 0.065   # +1.5% absolute
    converted = np.where(group == 'A',
                         np.random.binomial(1, conv_rate_A, n),
                         np.random.binomial(1, conv_rate_B, n))

    # Time on site (seconds)
    time_A = np.random.normal(120, 30, n)        # mean 120, sd 30
    time_B = np.random.normal(130, 35, n)        # mean 130, sd 35
    time_on_site = np.where(group == 'A', time_A, time_B)
    time_on_site = np.clip(time_on_site, 0, None)  # no negative times

    # Pages viewed
    pages_A = np.random.poisson(4, n)             # mean 4
    pages_B = np.random.poisson(4.3, n)           # mean 4.3
    pages_viewed = np.where(group == 'A', pages_A, pages_B)

    # Create DataFrame
    df = pd.DataFrame({
        'visitor_id': range(1, n+1),
        'group': group,
        'converted': converted,
        'time_on_site': np.round(time_on_site, 1),
        'pages_viewed': pages_viewed
    })

    df.to_csv(csv_path, index=False)
    print("✅ Saved as:", csv_path)

print(df.head())


# 2) Hypotheses 

# Conversion rate:
# Null hypothesis (H0): p_B <= p_A (B is not better than A)
# Alternative hypothesis (H1): p_B > p_A (B is better than A)
#   H0: p_B <= p_A
#   H1: p_B  > p_A
#
# Time on site:
#   H0: mu_B <= mu_A
#   H1: mu_B  > mu_A
#
# Pages viewed:
#   H0: mu_B <= mu_A
#   H1: mu_B  > mu_A


# 3) Summary Statistics
summary = df.groupby("group").agg(
    n=("visitor_id","count"),
    conv_rate=("converted","mean"),
    conversions=("converted","sum"),
    time_mean=("time_on_site","mean"),
    pages_mean=("pages_viewed","mean")
)
print("\n=== Summary (A vs B) ===")
print(summary)

# 4) Assumptions for Conversion (Two-proportion z-test)


assumptions = summary[["n","conversions"]].copy()
assumptions["failures"] = assumptions["n"] - assumptions["conversions"]
assumptions["successes_ok"] = assumptions["conversions"] >= 10
assumptions["failures_ok"] = assumptions["failures"] >= 10

print("\n=== Conversion Assumptions Check ===")
print(assumptions)



# 5) Conversion Rate Test (Two-proportion z-test, one-tailed)

conv = df.groupby("group")["converted"].agg(["sum","count"])

count = np.array([conv.loc["B","sum"], conv.loc["A","sum"]])       # conversions in B, A
nobs  = np.array([conv.loc["B","count"], conv.loc["A","count"]])   # sample sizes in B, A

z_stat, p_value_conv = proportions_ztest(count=count, nobs=nobs, alternative="larger")

pA = summary.loc["A","conv_rate"]
pB = summary.loc["B","conv_rate"]
diff_conv = pB - pA

print("\n=== Conversion z-test (one-tailed: B > A) ===")
print("pA =", pA)
print("pB =", pB)
print("diff (B-A) =", diff_conv)
print("z_stat =", z_stat)
print("p_value =", p_value_conv)


# 6) Time on Site Test (Welch t-test, one-tailed)

A_time = df[df["group"]=="A"]["time_on_site"]
B_time = df[df["group"]=="B"]["time_on_site"]

t_time, p_two_time = stats.ttest_ind(B_time, A_time, equal_var=False)
p_one_time = p_two_time/2 if t_time > 0 else 1 - (p_two_time/2)

diff_time = B_time.mean() - A_time.mean()

print("\n=== Time on Site Welch t-test (one-tailed: B > A) ===")
print("mean(A) =", A_time.mean())
print("mean(B) =", B_time.mean())
print("diff (B-A) =", diff_time)
print("t_stat =", t_time)
print("p_value =", p_one_time)


# 7) Pages Viewed Test (Welch t-test, one-tailed)

A_pages = df[df["group"]=="A"]["pages_viewed"]
B_pages = df[df["group"]=="B"]["pages_viewed"]

t_pages, p_two_pages = stats.ttest_ind(B_pages, A_pages, equal_var=False)
p_one_pages = p_two_pages/2 if t_pages > 0 else 1 - (p_two_pages/2)

diff_pages = B_pages.mean() - A_pages.mean()

print("\n=== Pages Viewed Welch t-test (one-tailed: B > A) ===")
print("mean(A) =", A_pages.mean())
print("mean(B) =", B_pages.mean())
print("diff (B-A) =", diff_pages)
print("t_stat =", t_pages)
print("p_value =", p_one_pages)


# 8) Final Decisions (alpha=0.05)
alpha = 0.05

print("\n=== FINAL DECISIONS (alpha=0.05) ===")
print("Conversion: Reject H0?" , p_value_conv < alpha)
print("Time on site: Reject H0?", p_one_time < alpha)
print("Pages viewed: Reject H0?", p_one_pages < alpha)

✅ Loaded: ab_test_data.csv
   visitor_id group  converted  time_on_site  pages_viewed
0           1     A          0         101.1             3
1           2     B          0         157.6             4
2           3     B          0         128.5             2
3           4     B          0          87.6             3
4           5     A          0         109.7             3

=== Summary (A vs B) ===
          n  conv_rate  conversions   time_mean  pages_mean
group                                                      
A      5076   0.053191          270  119.756994    4.046099
B      4924   0.063769          314  130.158306    4.322908

=== Conversion Assumptions Check ===
          n  conversions  failures  successes_ok  failures_ok
group                                                        
A      5076          270      4806          True         True
B      4924          314      4610          True         True

=== Conversion z-test (one-tailed: B > A) ===
pA = 0.0531914893617